In [2]:
import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import pandas as pd
import numpy as np
from pathlib import Path
import datetime
import sys
import json
from sklearn.model_selection import train_test_split
import tensorflow as tf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from data_extraction import *
from data_iterator import *
import self_correcting_regression
# import multi_layer_lstm

Unable to connect to the database


In [4]:
features_json_data = open('./features.json').read()
features = json.loads(features_json_data)

In [5]:
allTimeSeries = normalizeFeatures(None)
print("Normalized all features")

Normalized all features


In [6]:
allTimeSeries[9]

,icustay_id,hadm_id,age,gender,Heart Rate,Respiratory Rate,SpO2/SaO2,pH,Potassium,Calcium,...,Creatinine,Blood Pressure,1 hours urine output,6 hours urine output,Next 6 hours urine output,AKI,gcs,ventilation,vasoactive medications,sedative medications
Time,,,,,,,,,,,,,,,,,,,,,
2186-01-20 10:00:00,200034,145866,0.527778,0.0,0.573034,0.432432,1.0,0.996021,0.546875,0.824074,...,0.057471,0.861111,0.034936,0.0,0.0,1,1.0,0.0,0.0,0.0
2186-01-20 11:00:00,200034,145866,0.527778,0.0,0.573034,0.432432,1.0,0.996021,0.546875,0.824074,...,0.057471,0.861111,0.034936,0.0,1100.0,0,1.0,0.0,0.0,0.0
2186-01-20 12:00:00,200034,145866,0.527778,0.0,0.573034,0.432432,1.0,0.992042,0.562500,0.824074,...,0.057471,0.861111,0.034936,0.0,1500.0,0,1.0,0.0,0.0,0.0
2186-01-20 13:00:00,200034,145866,0.527778,0.0,0.573034,0.432432,1.0,0.992042,0.546875,0.824074,...,0.057471,0.861111,0.034936,0.0,1850.0,0,1.0,0.0,0.0,0.0
2186-01-20 14:00:00,200034,145866,0.527778,0.0,0.573034,0.432432,1.0,0.992042,0.546875,0.824074,...,0.057471,0.861111,0.034936,0.0,2200.0,0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186-01-23 06:00:00,200034,145866,0.527778,0.0,0.325843,0.351351,1.0,0.985411,0.562500,0.833333,...,0.045977,0.571429,0.036441,970.0,870.0,0,0.8,0.0,1.0,0.0
2186-01-23 07:00:00,200034,145866,0.527778,0.0,0.325843,0.459459,1.0,0.985411,0.562500,0.833333,...,0.045977,0.753968,0.034936,820.0,760.0,0,0.8,0.0,1.0,0.0
2186-01-23 08:00:00,200034,145866,0.527778,0.0,0.376404,0.405405,1.0,0.985411,0.562500,0.833333,...,0.045977,0.706349,0.037947,645.0,760.0,0,0.8,0.0,1.0,0.0


In [7]:
# allTimeSeries

In [8]:
train, test = train_test_split(allTimeSeries, test_size=0.2, random_state=12)

In [9]:
iterator = DataIterator(train)

In [10]:
x, y, sequenceLengths, musks = iterator.next_batch(256)

In [19]:
x.shape

(256, 12, 28)

In [21]:
tf.reset_default_graph()

In [ ]:
graph = self_correcting_regression.buildGraph(stateSizes=(512, 512), numFeatures=26)

In [18]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
# sess = tf.Session()
sess.run(tf.global_variables_initializer())
self_correcting_regression.trainGraph(graph, sess, train, test, learningRate=1e-6)

ValueError: Cannot feed value of shape (256, 12, 26) for Tensor 'Placeholder:0', which has shape '(?, ?, 28)'

In [ ]:
feed = {graph['x']: x, graph['y']: y, graph['seqlen']: sequenceLengths, graph['musk']: musks}

In [ ]:
sess.run(graph['cost'], feed_dict=feed)